# GPT Researcher



## Introduction

A hot area in Agentic AI is the wave of "Deep Research" AI tools offered by Google, OpenAI, Anthropic, Perplexity, etc. These AI Agents make it easy to implement the sort of system that I wrote in Python using the `instructor` package, Langchain, and the OpenAI models back in 2023.  Now, though, I'm experimenting with the open source tool [**GPT Researcher**](https://gptr.dev/), available as a Python package from [their Github repository](https://github.com/assafelovic/gpt-researcher) and deployable as a Model Context Protocol (MCP) server -- for example, [at the MCP Market](https://mcpmarket.com/server/gpt-researcher-1).

Here, I apply GPT Researcher both as an interactive web app and as an automated capability. The automated form allows me to more easily build a small pipeline around it so as to enforce ["intellectual honesty"](https://en.wikipedia.org/wiki/Intellectual_honesty) when analyzing more controversial topics.

## Install

We'll clone it straight from its Github repository.

In [1]:
!git clone https://github.com/assafelovic/gpt-researcher.git

Cloning into 'gpt-researcher'...
remote: Enumerating objects: 24790, done.
remote: Counting objects: 100% (184/184), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 24790 (delta 121), reused 78 (delta 73), pack-reused 24606 (from 3)
Receiving objects: 100% (24790/24790), 34.10 MiB | 6.77 MiB/s, done.
Resolving deltas: 100% (17521/17521), done.


In [5]:
%cd gpt-researcher

/content/gpt-researcher


In [6]:
!pip install -r requirements.txt

## Setup API Keys for LLMs, Web Scraping; and Folder for Custom Documents

We'll setup the subdirectory `/content/information` where we'll store custom documents, some of which to add domain-specific information and others to provide instructions altering the LLM behavior.  

This latter purpose is what the document `intellectual_honesty.md` does. It was generated by Perplexity with OpenAI's `o4-mini`. So, I manually upload it.

**NOTE: This upload step pauses the run for user input. So, if it
seems to be taking a while, check the output for the `Choose Files` interactive prompt waiting for your input.**

In [7]:
from google.colab import files, userdata
import os
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['TAVILY_API_KEY'] = userdata.get('TAVILY_API_KEY')

## Functions `get_research_report()`, `generate_detailed_report()`, `run_thorough_analyses()`

In [8]:
from json import load
from gpt_researcher import GPTResearcher
import asyncio
from typing import List
import datetime
from gpt_researcher.utils.enum import Tone
from backend.report_type import DetailedReport
import os
import re


# Function to perform a quick research analysis and return a Markdown report
async def get_research_report(query: str, report_type: str, report_source: str,context_docs: List=[],intl_hnsty: bool = True,verbose: bool = True) -> str:
    if intl_hnsty:
        docs = ["intellectual_honesty.md"]+context_docs
    else:
        docs = context_docs

    researcher = GPTResearcher(
        query=query,
        report_type=report_type,
        report_source=report_source,
        documents=docs,
        complement_source_urls=True,
        #source_urls=["https://en.wikipedia.org/wiki/Intellectual_honesty","https://en.wikipedia.org/wiki/List_of_fallacies"],
        verbose=verbose
    )
    research = await researcher.conduct_research()
    report = await researcher.write_report()
    return report

# Function to perform a "Deep Research" analysis and return a Markdown report
async def generate_detailed_report(query: str, context_docs: List=[]):
    if context_docs:
        also = f" (Consider these douments: {', '.join(context_docs)}.)"
    else:
        also = ""
    detailed_report = DetailedReport(
        query=query + also,
        report_type="research_report",
        report_source="hybrid",
        tone=Tone.Objective,
        complement_source_urls=True
    )

    final_report = await detailed_report.run()
    return final_report

# Function to apply a 3-phase intellectually honest analysis and return a dictionary with a Markdown report for each phase
async def run_thorough_analyses(proposition: str, base_name: str, directory: str='/content/information',context_docs:List=[], verbose: bool = True,force: bool = False,phases:int=1,download=True):
  # Get the current timestamp
  timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")

  # Construct the new filenames with the timestamp
  base_report = f"report_{base_name}_{timestamp}.md"
  critique = f"report_{base_name}_intel_honesty_{timestamp}.md"
  rev_report = f"report_{base_name}_rev_{timestamp}.md"

  report_source = "hybrid"
  report0 = None
  report2 = None
  report3 = None

  matching_files = [f for f in os.listdir(directory) if re.match(rf"report_{base_name}_[^_]+\.md", f)]
  if (phases>0) and (not matching_files or force):
    query = f"Assess proposition '{proposition}' (leverage document 'intellectual_honesty.md')"
    report0 = await get_research_report(query=query, report_type="research_report", report_source=report_source, context_docs=context_docs)
    with open(directory + f'/{base_report}', 'w') as f:
      f.write(report0)


  matching_files1 = [f for f in os.listdir(directory) if re.match(rf"report_{base_name}_[^_]+\.md", f)]
  if matching_files1 and (base_report not in matching_files1):
    base_report = matching_files1[-1]
    # Read in report0 from file base_report
    with open(directory + f'/{base_report}', 'r') as f:
      report0 = f.read()


  matching_files2 = [f for f in os.listdir(directory) if re.match(rf"report_{base_name}_intel_honesty_[^_]+\.md", f)]
  if (phases>1) and matching_files1 and (not matching_files2 or force):
    #query2 = f"""
    #Analyze the reference sources and conclusions used in the analysis of '{proposition}' in document "{base_report}" for intellectual honesty (per document "intellectual_honesty.md"), esp. for balance, fallacies, bias, exhaustiveness of hypotheses, selectivity of evidence, historical context, and factual accuracy.
    #"""
    #report2 = await get_research_report(query=query2, report_type="research_report", report_source=report_source,context_docs=[base_report]+context_docs,intl_hnsty=True)

    query2 = f"""Adhering to 'rubric_assessment_intel_honesty_2.md', grade the analysis of '{proposition}' in document "{base_report}" for intellectual honesty."""
    report2 = await get_research_report(query=query2, report_type="research_report", report_source=report_source,context_docs=[base_report]+context_docs,intl_hnsty=False)

    with open(directory + f'/{critique}', 'w') as f:
      f.write(report2)

  matching_files2 = [f for f in os.listdir(directory) if re.match(rf"report_{base_name}_intel_honesty_[^_]+\.md", f)]
  if matching_files2 and (critique not in matching_files2):
    critique = matching_files2[-1]
    # Read in report2 from file critique
    with open(directory + f'/{critique}', 'r') as f:
      report2 = f.read()

  matching_files3 = [f for f in os.listdir(directory) if re.match(rf"report_{base_name}_rev_[^_]+\.md", f)]
  if (phases>2) and matching_files1 and matching_files2 and (not matching_files3 or force):
    query3 = f"""
Improve upon the analysis of '{proposition}' in document "{base_report}" given the critique and suggestions in document "{critique}". (Focus output content on the proposition itself rather than the methodology.)
  """
    report3 = await generate_detailed_report(query=query3,context_docs=context_docs)
    with open(directory + f'/{rev_report}', 'w') as f:
      f.write(report3)
  matching_files3 = [f for f in os.listdir(directory) if re.match(rf"report_{base_name}_rev_[^_]+\.md", f)]
  if matching_files3 and (rev_report not in matching_files3):
    rev_report = matching_files3[-1]
    # Read in report3 from file rev_report
    with open(directory + f'/{rev_report}', 'r') as f:
      report3 = f.read()

  if download:
    # List all files in the directory
    for filename in os.listdir(directory):
        # Check if the filename starts with the base name
        if filename.startswith(f"report_{base_name}"):
            # Construct the full file path
            file_path = os.path.join(directory, filename)
            # Download the file (assuming files.download is available)
            try:
                files.download(file_path)
            except NameError:
                print(f"Error: files.download not found.  File '{filename}' not downloaded.  Please ensure you're in a Google Colab environment.")
            except Exception as e:
                print(f"An error occurred while downloading {filename}: {e}")

  return {"base_report": report0, "critique": report2, "rev_report": report3}


### R Markdown Report

In [9]:
import os
import re
import datetime
import shutil
from google.colab import files


# prompt: create a general expression that extracts a suitably concise title from a markdown document results["rev_report"]
def extract_title(markdown_text):
  """
  Extracts a concise title from a markdown document.

  Args:
    markdown_text: The content of the markdown document as a string.

  Returns:
    A string representing the extracted title, or None if no suitable title is found.
  """
  # Use a regular expression to find lines starting with "# " (level 1 heading)
  match = re.search(r"^# (.*)", markdown_text, re.MULTILINE)
  if match:
    return match.group(1).strip()

  # If no level 1 heading is found, try level 2 headings
  match = re.search(r"^## (.*)", markdown_text, re.MULTILINE)
  if match:
      return match.group(1).strip()

  # If no suitable heading is found, return None
  return None

# prompt: use package `re` to get the starting substring of the `title` string before the ':', if any, i.e. the main title dropping the subtitle.
def extract_main_title(title):
  """
  Extracts the main title from a string, removing any subtitle after a colon.
  """
  match = re.search(r"^(.*?):", title)
  if match:
    return match.group(1).strip()
  return title

# prompt: a function that takes a markdown document (string) as input and outputs that document with all header line (starting with regex '^#') changed to "##"; i.e. demote every section one level.
import re

def demote_headers(markdown_text):
  """Demotes all headers in a markdown string by one level."""
  return re.sub(r'^#', '##', markdown_text, flags=re.MULTILINE)



def download_report_rmd(proposition, base_name, directory, results, date_string = ''):

    if not date_string:
      date_string = datetime.datetime.now().strftime("%Y-%m-%d")

    new_filename = f"gpt_researcher_{base_name}_rev.pdf"
    new_rmd_filename = f"gpt_researcher_{base_name}_rev.Rmd"
    new_filepath = os.path.join(directory, new_filename)

    # Consolidated final report with the following structure:
    report_title = extract_title(results["rev_report"])
    short_title = extract_main_title(report_title)

    yaml_rmd = f"""---
title: "GPT Researcher:  {report_title}"
author: "Michael L. Thompson"
date: "{date_string}"
output:
  pdf_document:
    latex_engine: xelatex
    toc: yes
    toc_depth: 4
linkcolor: red
urlcolor: blue
---

^['GPT Researcher: {short_title}' [{new_filename}] © {date_string[:4]} by Michael L. Thompson, generated using [GPT-Researcher](https://github.com/assafelovic/gpt-researcher), is licensed under CC BY-SA 4.0. To view a copy of this license, visit http://creativecommons.org/licenses/by-sa/4.0/]


\\newpage
"""


    appdx_header = """
\\newpage

# APPENDIX

## Central Proposition

The final report above was generated by GPT Researcher using the "Deep Research" analysis by the `gpt_researcher` object `DetailedReport` given the following prompt:

> `f"Improve upon the analysis of '{proposition}' in document "{base_report}" given the critique and suggestions in document "{critique}". (Focus output content on the proposition itself rather than the methodology.)"`

Where the **Central Proposition** was the following:

* **`proposition =  """ + f'"{proposition}"' + """`**

Below are the results of the Phase 1 analysis, `base_report`, and the Phase 2 analysis, `critique`. Both of them were quick research reports using the `gpt_researcher` object `GPTResearcher` and the following prompts, respectively:

* **Phase 1:** `f"Assess proposition '{proposition}' (leverage document 'intellectual_honesty.md')"`

* **Phase 2:** `f'Analyze the reference sources and conclusions used in the analysis of '{proposition}' in document "{base_report}" for intellectual honesty (per document "intellectual_honesty.md"), esp. for balance, fallacies, bias, exhaustiveness of hypotheses, selectivity of evidence, historical context, and factual accuracy.'`

    """


    report_template = f"""{yaml_rmd}

{results["rev_report"]}

{appdx_header}


## Phase 1, `base_report`

{demote_headers(results["base_report"])}


## Phase 2, `critique`

{demote_headers(results["critique"])}
"""

    # Construct the full file path
    file_path = os.path.join(directory, new_rmd_filename)
    with open(file_path, 'w') as fl:
      fl.write(report_template)
    # Download the file (assuming files.download is available)
    try:
        files.download(file_path)
    except NameError:
        print(f"Error: files.download not found.  File '{new_rmd_filename}' not downloaded.  Please ensure you're in a Google Colab environment.")
    except Exception as e:
        print(f"An error occurred while downloading {new_rmd_filename}: {e}")

    return report_template

def move_reports(directory = "/content/information",destination_directory = "/content/storage"):

    # Create the destination directory if it doesn't exist
    if not os.path.exists(destination_directory):
        os.makedirs(destination_directory)
    moved_files = []
    # List all files in the source directory
    for filename in os.listdir(directory):
        # Check if the filename starts with "report_"
        if filename.startswith("report_") or filename.startswith("gpt_researcher_"):
            # Construct the full source and destination paths
            source_path = os.path.join(directory, filename)
            destination_path = os.path.join(destination_directory, filename)
            # Move the file
            shutil.move(source_path, destination_path)
            moved_files.append(filename)
            print(f"Moved '{filename}' to '{destination_directory}'")
    return moved_files





In [ ]:
from google.colab import files
import re
import os


def download_full_report_md(full_report_rmd: str, base_name: str, directory: str):
    """ Takes the full report as R Markdown (Rmd) and strips off the YAML header to make it plain markdown (md) and downloads it to user."""

    file_path_to_download = os.path.join(directory, f'gpt_researcher_{base_name}_rev.md')

    full_report_rmd_wo_yaml = re.sub(r'^--- *$.+?^\\newpage *$','',full_report_rmd,flags= re.DOTALL | re.MULTILINE)

    # Write the string to the file
    with open(file_path_to_download, 'w') as f:
        f.write(full_report_rmd_wo_yaml)

    try:
        files.download(file_path_to_download)
    except NameError:
        print(f"Error: files.download not found. File '{file_path_to_download}' not downloaded. Please ensure you're in a Google Colab environment.")
    except FileNotFoundError:
        print(f"Error: File not found at '{file_path_to_download}'.")
    except Exception as e:
        print(f"An error occurred while downloading {file_path_to_download}: {e}")

    return (file_path_to_download, full_report_rmd_wo_yaml)

## **Upload `intellectual_honesty.md`**

In [10]:
# prompt: make the directory /content/information and prompt the user to upload documents to it

import os
from google.colab import files


directory = "/content/information"
os.environ['DOC_PATH'] = directory

#SKIP: use following code chunk instead
if False:
    # Create the directory if it doesn't exist
    if not os.path.exists(directory):
        os.makedirs(directory)

    # Prompt the user to upload files
    uploaded = files.upload()

    # Move the uploaded files to the specified directory
    for filename in uploaded.keys():
        os.rename(filename, os.path.join(directory, filename))

    print(f"Files uploaded to: {directory}")

# prompt: given a sharable url for a file on google drive, copy it to `directory`
import os
import re
import requests

def download_gdrive_file(url, destination_directory):
  """
  Downloads a file from a public Google Drive sharing URL to a specified directory.

  Args:
    url: The sharable URL of the Google Drive file.
    destination_directory: The local directory to save the file.
  """
  # Extract the file ID from the URL
  file_id = url.split('/d/')[1].split('/')[0]

  # Construct the download URL
  download_url = f'https://drive.google.com/uc?export=download&id={file_id}'

  # Ensure the destination directory exists
  if not os.path.exists(destination_directory):
    os.makedirs(destination_directory)

  # Download the file
  try:
    response = requests.get(download_url, stream=True)
    response.raise_for_status()  # Raise an exception for bad status codes

    # Get the filename from the response headers or infer it
    # This is a simplified approach; a more robust method might parse Content-Disposition
    filename = url.split('/')[-1]
    if 'content-disposition' in response.headers:
        d = response.headers['content-disposition']
        fname_match = re.findall("filename\*?=[\'\"]?(?:UTF-\d+\'\')?([^\'\"]*)[\'\"]?", d)
        if fname_match:
            filename = fname_match[0]


    destination_path = os.path.join(destination_directory, filename)

    with open(destination_path, 'wb') as f:
      for chunk in response.iter_content(chunk_size=8192):
        f.write(chunk)

    print(f"Downloaded '{filename}' to '{destination_directory}'")
  except requests.exceptions.RequestException as e:
    print(f"Error downloading file: {e}")

# Example usage:
# Replace with your actual Google Drive sharable URL and desired directory
# gdrive_url = 'YOUR_GOOGLE_DRIVE_SHARABLE_URL'
# target_directory = '/content/your_target_directory'
# download_gdrive_file(gdrive_url, target_directory)

# URL to `intellectual_honesty.md`
gdrive_url1 = 'https://drive.google.com/file/d/1FVyGtReG1SU89p3lTO6jP_IBsPXiQK4-/view?usp=sharing' # intellectual_honesty.md
directory = '/content/information'
download_gdrive_file(gdrive_url1, directory)
gdrive_url2 = 'https://drive.google.com/file/d/1WpNQBKZk5pEjlN6Ep7wK0KEv_VA-24f7/view?usp=sharing' # bayesian_confirmation.md
download_gdrive_file(gdrive_url2, directory)
gdrive_url3 ='https://drive.google.com/file/d/1hzTH4CKCXge2kRWTcctlcKNy7xdtr001/view?usp=sharing' # Bayesian Process Tracing_ AI Agent Instruction Man.md
download_gdrive_file(gdrive_url3, directory)
#gdrive_url4 ='https://drive.google.com/file/d/1XGx2Di_M6IzDRDBz3e24dWtP1sApXkw_/view?usp=sharing' # rubric_assessment_intel_honesty.md
gdrive_url4 ='https://drive.google.com/file/d/1Lps1j-W3XOCwTHo025GoiqRMPujbSaED/view?usp=sharing' # rubric_assessment_intel_honesty_2.md
download_gdrive_file(gdrive_url4, directory)

Downloaded 'intellectual_honesty.md' to '/content/information'
Downloaded 'bayesian_confirmation.md' to '/content/information'
Downloaded 'Bayesian Process Tracing_ AI Agent Instruction Man.md' to '/content/information'
Downloaded 'rubric_assessment_intel_honesty_2.md' to '/content/information'


# Try GPT Researcher Programmatically (by code)

In [11]:
proposition = "Tom Brady is the NFL GOAT."
base_name = "brady"

results = await run_thorough_analyses(proposition=proposition, base_name=base_name, phases=3,context_docs=['bayesian_confirmation.md','Bayesian Process Tracing_ AI Agent Instruction Man.md','rubric_assessment_intel_honesty_2.md'])
full_report_rmd = download_report_rmd(proposition=proposition, base_name=base_name, directory=directory, results=results)
fname_full_report_md, full_report_rmd_wo_yaml = download_full_report_md(full_report_rmd=full_report_rmd, base_name=base_name, directory=directory)

INFO:     [12:43:05] 🔍 Starting the research task for 'Assess proposition 'Tom Brady is the NFL GOAT.' (leverage document 'intellectual_honesty.md')'...
INFO:     [12:43:05] 🏈 Sports Analyst Agent
INFO:     [12:43:12] 🌐 Browsing the web to learn more about the task: Assess proposition 'Tom Brady is the NFL GOAT.' (leverage document 'intellectual_honesty.md')...
INFO:     [12:43:14] 🤔 Planning the research strategy and subtasks...
INFO:     [12:43:21] 🗂️ I will conduct my research based on the following queries: ['Objective criteria NFL GOAT quarterback evaluation statistical contextual analysis', 'Balanced evidence Tom Brady GOAT debate championships team impact analysis', 'Intellectual honesty framework sports debate methodology Tom Brady GOAT', "Assess proposition 'Tom Brady is the NFL GOAT.' (leverage document 'intellectual_honesty.md')"]...
INFO:     [12:43:21] 
🔍 Running research for 'Objective criteria NFL GOAT quarterback evaluation statistical contextual analysis'...
INFO:     

# Assessing the Proposition: "Tom Brady is the NFL GOAT"


## Introduction


The proposition "Tom Brady is the NFL GOAT" (Greatest of All Time) is one of the most debated topics in American sports discourse. This report systematically evaluates the claim using the intellectual honesty framework outlined in *intellectual_honesty.md*, emphasizing full disclosure, consistent standards, openness to revision, and engagement with alternatives. The analysis draws on a comprehensive review of recent, reliable sources, including statistical databases, sports journalism, and expert commentary, to provide a balanced, evidence-based conclusion.


## 1. Defining the GOAT Criteria


### 1.1. What Constitutes "GOAT" in the NFL?


The term "GOAT" is inherently subjective, but common criteria include:


- **Championships and Playoff Success**

- **Longevity and Consistency**

- **Statistical Production (Cumulative and Rate Stats)**

- **Awards and Honors**

- **Impact on Team Success**

- **Individual 

INFO:     [12:44:47] 📝 Report written for 'Assess proposition 'Tom Brady is the NFL GOAT.' (leverage document 'intellectual_honesty.md')'


- intellectual_honesty.md

- rubric_assessment_intel_honesty_2.md


INFO:     [12:44:49] 🔍 Starting the research task for 'Adhering to 'rubric_assessment_intel_honesty_2.md', grade the analysis of 'Tom Brady is the NFL GOAT.' in document "report_brady_20250531124258.md" for intellectual honesty.'...
INFO:     [12:44:49] 🧑‍⚖️ Academic Integrity Agent
INFO:     [12:44:50] 🌐 Browsing the web to learn more about the task: Adhering to 'rubric_assessment_intel_honesty_2.md', grade the analysis of 'Tom Brady is the NFL GOAT.' in document "report_brady_20250531124258.md" for intellectual honesty....
INFO:     [12:44:54] 🤔 Planning the research strategy and subtasks...


...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

NameError: name 'download_full_report_md' is not defined